# Global run (finance overrides via interest-rate CSV)

Runs the green ammonia optimisation over either:
- all locations in `data/20251222_land_max_capacity.csv`, or
- a user-provided list of `(lat, lon)` pairs.

This notebook additionally applies per-location, per-technology financing overrides from a CSV with columns: `lat, lon, tech, interest_rate`.

Outputs:
- `results/global_run_finance_overrides_results.csv` (one row per location)
- `results/global_lcoa_heatmap_finance_overrides.html` (interactive)

Tip: in VS Code, select the `.venv` interpreter (or a kernel that points at `.venv/bin/python`) so notebook dependencies match the repo.

In [1]:
import os
import sys
from pathlib import Path

import pandas as pd
import plotly.io as pio

# Make repo importable
repo_root = Path().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from model import plot_global_heatmap
from model import run_global

OUTPUT_DIR = repo_root / 'results'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Prefer non-interactive renderers unless running in a notebook UI that supports it
pio.renderers.default = pio.renderers.default or 'notebook'

In [2]:
# ----------------
# USER PARAMETERS
# ----------------
# If LOCATIONS is None: run all locations from the land-availability CSV.
# Otherwise: provide a list like [(lat1, lon1), (lat2, lon2), ...].
LOCATIONS = None
# LOCATIONS = [(-23.0, 133.0), (51.0, 0.0)]

# Optional path to a CSV with columns: lat, lon (overrides LOCATIONS if set)
LOCATIONS_CSV = None  # e.g. repo_root / 'inputs' / 'my_locations.csv'

# Finance overrides CSV (lat, lon, tech, interest_rate)
INTEREST_CSV = repo_root / 'inputs' / 'example_finance_overrides.csv'

# Runtime controls
AGGREGATION_COUNT = 1
TIME_STEP_H = 1.0
MAX_SNAPSHOTS = 168  # e.g. 168 for quick tests; None = full series
QUIET = True

# Where to save aggregated outputs
RESULTS_CSV = OUTPUT_DIR / 'global_run_finance_overrides_results.csv'
RESULTS_XLSX = OUTPUT_DIR / 'global_run_finance_overrides_results.xlsx'

# Heatmap outputs
HEATMAP_HTML = OUTPUT_DIR / 'global_lcoa_heatmap_finance_overrides.html'
COLOR_COLUMN = 'lcoa_usd_per_t'
COLOR_SCALE = 'Cividis'

In [3]:
# 10x10 contiguous 1° grid starting at (-23, 133) for testing
BASE_LAT = -23.0
BASE_LON = 133.0
LOCATIONS = [(BASE_LAT + i, BASE_LON + j) for i in range(10) for j in range(10)]

In [4]:
def load_locations_from_csv(path: str | Path):
    path = Path(path)
    df = pd.read_csv(path)
    if 'lat' not in df.columns or 'lon' not in df.columns:
        raise ValueError(f'Locations CSV must have columns lat, lon: {path}')
    return list(zip(df['lat'].astype(float), df['lon'].astype(float)))

if LOCATIONS_CSV is not None:
    locations = load_locations_from_csv(LOCATIONS_CSV)
else:
    locations = LOCATIONS

print('Locations mode:', 'ALL (from land CSV)' if locations is None else f'{len(locations)} explicit locations')
print('Interest overrides CSV:', INTEREST_CSV)

Locations mode: 100 explicit locations
Interest overrides CSV: /Users/carlopalazzi/programming/pypsa_models/green-lory/inputs/example_finance_overrides.csv


In [5]:
# Run the batch optimisation.
# - Progress bar is printed while looping.
# - QUIET=True suppresses solver/pypsa chatter for each run.
results_df = run_global.run_global(
    locations=locations,
    interest_csv=INTEREST_CSV,
    aggregation_count=AGGREGATION_COUNT,
    time_step=TIME_STEP_H,
    max_snapshots=MAX_SNAPSHOTS,
    output_csv=RESULTS_CSV,
    quiet=QUIET,
 )

print(f'Finished: {len(results_df)} locations')
print('Saved CSV to:', RESULTS_CSV)

display(results_df.head())

[############################] 100/100 (100.0%)     DONE lat= -14.00 lon= 142.00


Finished: 100 locations
Saved CSV to: /Users/carlopalazzi/programming/pypsa_models/green-lory/results/global_run_finance_overrides_results.csv


,latitude,longitude,country,lcoa_usd_per_t,annual_ammonia_demand_mwh,annual_ammonia_production_t,total_cost_usd_per_year,cost_share_wind_pct,lcoa_component_wind_usd_per_t,cost_share_solar_pct,...,battery_mwh,accumulated_penalty_mwh,hydrogen_storage_capacity_t,land_capacity_cap_mw,land_onshore_pct,land_cell_area_km2,area_cap_mw,solar_area_used_km2,wind_area_used_km2,interest_overrides_applied
-23.0_133.0,-23.0,133.0,Australia,572.736435,6.250000e+07,1.000000e+07,5.727364e+09,39.640276,227.034304,23.776666,...,10.444564,0.0,774.064123,919829.415574,100.0,11422.989537,919829.415574,11422.989537,5661.719092,True
-23.0_134.0,-23.0,134.0,Australia,610.122766,6.250000e+07,1.000000e+07,6.101228e+09,32.120641,195.975340,27.568299,...,29.169970,0.0,1297.101184,919666.637973,100.0,11422.989537,919666.637973,11422.989537,5629.163572,True
-23.0_135.0,-23.0,135.0,Australia,586.069214,6.250000e+07,1.000000e+07,5.860692e+09,36.606663,214.540383,23.511037,...,5.792049,0.0,1287.775301,919571.113223,100.0,11422.989537,919571.113223,11422.989537,5610.058622,True
-23.0_136.0,-23.0,136.0,Australia,531.932553,6.250000e+07,1.000000e+07,5.319326e+09,40.159902,213.623589,22.338717,...,0.000000,0.0,581.130209,920069.726716,100.0,11422.989537,920069.726716,11422.989537,5709.781320,True
-23.0_137.0,-23.0,137.0,Australia,511.724818,6.250000e+07,1.000000e+07,5.117248e+09,41.298757,211.335988,20.582523,...,0.000000,0.0,314.488987,920073.296401,100.0,11422.989537,920073.296401,11422.989537,5710.495257,True


In [6]:
# Plot choropleth heatmap of LCOA (grid-cell polygons)
fig = plot_global_heatmap.plot_lcoa_heatmap(
    RESULTS_CSV,
    color_column=COLOR_COLUMN,
    cell_size_deg=1.0,
    color_scale=COLOR_SCALE,
 )

# Inline display (nbformat is installed in the intended kernels now)
fig.show()

# Save interactive HTML
fig.write_html(HEATMAP_HTML)
print('Saved heatmap HTML to:', HEATMAP_HTML)

Saved heatmap HTML to: /Users/carlopalazzi/programming/pypsa_models/green-lory/results/global_lcoa_heatmap_finance_overrides.html
